# Sequential 모델

## 설정

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Sequential 모델을 사용하는 경우

`Sequential` 모델은 각 레이어에 **정확히 하나의 입력 텐서와 하나의 출력 텐서**가 있는 **일반 레이어 스택**에 적합합니다.

개략적으로 다음과 같은 `Sequential` 모델은

In [2]:
a = [layers.Dense(2),layers.Dense(3),layers.Dense(4)]
print(a)

[<keras.layers.core.dense.Dense object at 0x000002399B228F40>, <keras.layers.core.dense.Dense object at 0x0000023981DF5610>, <keras.layers.core.dense.Dense object at 0x0000023981DF51F0>]


In [5]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"), # (3,3)(3,2)+(2,)
        layers.Dense(3, activation="relu", name="layer2"), # (3,2)(2,3)+(3,)
        layers.Dense(4, name="layer3"),                    # (3,3)(3,4)+(4,)
                                                           # (3,4)
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
print(x.numpy())
y = model(x)
print(y.shape)
print(y)

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
(3, 4)
tf.Tensor(
[[ 0.16431224  0.16719037  0.08515974 -0.2233285 ]
 [ 0.16431224  0.16719037  0.08515974 -0.2233285 ]
 [ 0.16431224  0.16719037  0.08515974 -0.2233285 ]], shape=(3, 4), dtype=float32)


다음 함수와 동일합니다.

In [6]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1") # (3,3)(3,2)+(2,)
layer2 = layers.Dense(3, activation="relu", name="layer2") # (3,2)(2,3)+(3,)
layer3 = layers.Dense(4, name="layer3")                    # (3,3)(3,4)+(4,)
                                                           # (3,4)

# Call layers on a test input
x = tf.ones((3, 3))
print(x.shape)
x = layer1(x)
print(x.shape)
x = layer2(x)
print(x.shape)
y = layer3(x)
print(y.shape)
# y = layer3(layer2(layer1(x)))

(3, 3)
(3, 2)
(3, 3)
(3, 4)


Sequential 모델은 다음의 경우에 **적합하지 않습니다**.

- 모델에 다중 입력 또는 다중 출력이 있습니다
- 레이어에 다중 입력 또는 다중 출력이 있습니다
- 레이어 공유를 해야 합니다
- 비선형 토폴로지를 원합니다(예: 잔류 연결, 다중 분기 모델)

## Sequential 모델 생성하기

레이어의 목록을 Sequential 생성자에 전달하여 Sequential 모델을 만들 수 있습니다.

In [7]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)

속한 레이어는 `layers` 속성을 통해 접근할 수 있습니다.

In [8]:
dir(model)

['_SCALAR_UPRANKING_ON',
 '_TF_MODULE_IGNORED_PROPERTIES',
 '__call__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_trackable',
 '_add_trackable_child',
 '_add_variable_with_custom_getter',
 '_assert_compile_was_called',
 '_assert_weights_created',
 '_auto_track_sub_layers',
 '_autocast',
 '_autographed_call',
 '_base_model_initialized',
 '_build_graph_network_for_inferred_shape',
 '_build_input_shape',
 '_call_accepts_kwargs',
 '_call_arg_was_passed',
 '_call_fn_arg_defaults',
 '_call_fn_arg_positions',
 '_call_fn_args',
 '_call_full_argspec',
 '_callable_losses',
 '_ca

In [9]:
model.layers

`add()` 메서드를 통해 Sequential 모델을 점진적으로 작성할 수도 있습니다.

In [13]:
model = keras.Sequential()
print(model.layers)
model.add(layers.Dense(2, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))
print(model.layers)

[]
[<keras.layers.core.dense.Dense object at 0x0000023982003DC0>, <keras.layers.core.dense.Dense object at 0x0000023982003E50>, <keras.layers.core.dense.Dense object at 0x0000023981E71AF0>]


레이어를 제거하는 `pop()` 메서드도 있습니다. Sequential 모델은 레이어의 리스트와 매우 유사하게 동작합니다.

In [14]:
model.pop() # Last In First Out
print(len(model.layers))  # 2
print(model.layers)

2
[<keras.layers.core.dense.Dense object at 0x0000023982003DC0>, <keras.layers.core.dense.Dense object at 0x0000023982003E50>]


또한 Sequential 생성자는 Keras의 모든 레이어 또는 모델과 마찬가지로 `name` 인수를 허용합니다. 이것은 의미론적으로 유의미한 이름으로 TensorBoard 그래프에 주석을 달 때 유용합니다.

In [15]:
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2, activation="relu", name="layer1"))
model.add(layers.Dense(3, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))

## 미리 입력 형상 지정하기

일반적으로 Keras의 모든 레이어는 가중치를 만들려면 입력의 형상을 알아야 합니다. 따라서 다음과 같은 레이어를 만들면 처음에는 가중치가 없습니다.

In [16]:
layer = layers.Dense(3, activation="sigmoid") # (None,None)(None,3)+(3,)
layer.weights  # Empty

[]

가중치는 모양이 입력의 형상에 따라 달라지기 때문에 입력에서 처음 호출될 때 가중치를 만듭니다.

In [19]:
# Call layer on a test input
x = tf.ones((1, 4))
print(x)
y = layer(x)   # (1,4)(4,3)+(3,)
layer.weights  # Now it has weights, of shape (4, 3) and (3,)

tf.Tensor([[1. 1. 1. 1.]], shape=(1, 4), dtype=float32)


[<tf.Variable 'dense_12/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[ 0.6993152 , -0.52920866,  0.39725947],
        [-0.48262325, -0.06395864,  0.8785807 ],
        [-0.666992  ,  0.12566209, -0.65765107],
        [ 0.23606896,  0.48331237,  0.6424229 ]], dtype=float32)>,
 <tf.Variable 'dense_12/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [20]:
print(y)

tf.Tensor([[0.4466154  0.50393665 0.7791118 ]], shape=(1, 3), dtype=float32)


당연히 이것은 Sequential 모델에도 적용됩니다. 입력 형상이 없는 Sequential 모델을 인스턴스화할 때는 "빌드"되지 않습니다. 가중치가 없습니다(그리고 `model.weights`를 호출하면 오류가 발생함). 모델에 처음 입력 데이터가 표시되면 가중치가 생성됩니다.

In [23]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),  # (1,4)(4,2)+(2,)
        layers.Dense(3, activation="relu"),  # (1,2)(2,3)+(3,)
        layers.Dense(4),                     # (1,3)(3,4)+(4,)
                                             # (1,4)
    ]
)  # No weights at this stage!

# At this point, you can't do this:
# model.weights

# You also can't do this:
# model.summary()

# Call the model on a test input
x = tf.ones((1, 4))
y = model(x)
print("Number of weights after calling the model:", len(model.weights))  # 6
print(model.weights)

Number of weights after calling the model: 6
[<tf.Variable 'dense_19/kernel:0' shape=(4, 2) dtype=float32, numpy=
array([[-0.95969033,  0.07297444],
       [-0.9425423 , -0.12102175],
       [-0.8939574 ,  0.27549434],
       [-0.73153305, -0.9160409 ]], dtype=float32)>, <tf.Variable 'dense_19/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>, <tf.Variable 'dense_20/kernel:0' shape=(2, 3) dtype=float32, numpy=
array([[-0.1388393 ,  0.24829853, -1.0543964 ],
       [ 0.9449041 , -0.36035627, -0.6705326 ]], dtype=float32)>, <tf.Variable 'dense_20/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>, <tf.Variable 'dense_21/kernel:0' shape=(3, 4) dtype=float32, numpy=
array([[ 0.77778566, -0.3198616 , -0.7883109 , -0.4831347 ],
       [ 0.8845767 , -0.8880298 ,  0.16448891,  0.886657  ],
       [-0.08271652, -0.06831014, -0.6935117 , -0.21384764]],
      dtype=float32)>, <tf.Variable 'dense_21/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0

모델이 "빌드"되면, 그 내용을 표시하기 위해 `summary()` 메서드를 호출할 수 있습니다.

In [24]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (1, 2)                    10        
                                                                 
 dense_20 (Dense)            (1, 3)                    9         
                                                                 
 dense_21 (Dense)            (1, 4)                    16        
                                                                 
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


그러나 현재 출력 형상을 포함하여 지금까지 모델의 요약을 표시할 수 있도록 Sequential 모델을 점진적으로 빌드할 때 매우 유용할 수 있습니다. 이 경우 `Input` 객체를 모델에 전달하여 모델의 시작 형상을 알 수 있도록 모델을 시작해야 합니다.

In [27]:
a = (None, 4)
print(a)
print(len(a))

(None, 4)
2


In [26]:
print(keras.Input(shape=(4,)))

KerasTensor(type_spec=TensorSpec(shape=(None, 4), dtype=tf.float32, name='input_2'), name='input_2', description="created by layer 'input_2'")


In [25]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))             # (None,4)
model.add(layers.Dense(2, activation="relu"))  # (None,4)(4,2)+(2,)
                                               # (None,2)

model.summary()
model.weights

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


[<tf.Variable 'dense_22/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[-0.4999807 ,  0.40011144],
        [ 0.7251868 , -0.9697583 ],
        [-0.2247572 , -0.51448774],
        [ 0.9690201 , -0.80268383]], dtype=float32)>,
 <tf.Variable 'dense_22/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

`Input` 객체는 레이어가 아니므로 `model.layers`의 일부로 표시되지 않습니다.

In [28]:
model.layers

간단한 대안은 첫 번째 레이어에 `input_shape` 인수를 전달하는 것입니다.

In [29]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu", input_shape=(4,)))  # (None,4)(4,2)+(2,)

model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


이처럼 사전 정의된 입력 모양으로 빌드된 모델은 항상 가중치를 가지며(데이터를 보기 전에도) 항상 정의된 출력 형상을 갖습니다.

일반적으로 Sequential 모델의 입력 형상을 알고 있는 경우 항상 Sequential 모델의 입력 형상을 지정하는 것이 좋습니다.

## 일반적인 디버깅 워크플로우: `add()` + `summary()`

새로운 Sequential 아키텍처를 구축할 때는 `add()` 하여 레이어를 점진적으로 쌓고 모델 요약을 자주 인쇄하는 것이 유용합니다. 예를 들어 `Conv2D` 및 `MaxPooling2D` 레이어의 스택이 이미지 특성 맵을 다운 샘플링 하는 방법을 모니터링할 수 있습니다.

In [30]:
keras.Input(shape=(250, 250, 3))

<KerasTensor: shape=(None, 250, 250, 3) dtype=float32 (created by layer 'input_3')>

In [33]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images  (250-5)//2 + 1
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))  #  (None,250,250,3)(5,5,3,32)+(32,)
model.add(layers.Conv2D(32, 3, activation="relu"))             #  (None,123,123,32)(3,3,32,32)+(32,)
model.add(layers.MaxPooling2D(3))                              #  (None,121,121,32)
                                                               #  (None,40,40,32)

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()

# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu"))  #  (None,40,40,32)(3,3,32,32)+(32,)
model.add(layers.Conv2D(32, 3, activation="relu"))  #  (None,38,38,32)(3,3,32,32)+(32,)
model.add(layers.MaxPooling2D(3))                   #  (None,36,36,32)
model.add(layers.Conv2D(32, 3, activation="relu"))  #  (None,12,12,32)(3,3,32,32)+(32,)
model.add(layers.Conv2D(32, 3, activation="relu"))  #  (None,10,10,32)(3,3,32,32)+(32,)
model.add(layers.MaxPooling2D(2))                   #  (None,8,8,32)
                                                    #  (None,4,4,32)

# And now?
model.summary()

# # Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())              #  (None,4,4,32)

# # Finally, we add a classification layer.
model.add(layers.Dense(10))                         #  (None,32)(32,10)+(10,)
                                                    #  (None,10)
model.summary()    

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 123, 123, 32)      2432      
                                                                 
 conv2d_13 (Conv2D)          (None, 121, 121, 32)      9248      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 40, 40, 32)       0         
 2D)                                                             
                                                                 
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 123, 123, 32)      2432      
                          

매우 실용적이죠?


## 모델이 완성되면 해야 할 일

모델 아키텍처가 준비되면 다음을 수행할 수 있습니다.

- 모델을 훈련시키고 평가하며 추론을 실행합니다. [내장 루프를 사용한 훈련 및 평가 가이드](https://www.tensorflow.org/guide/keras/train_and_evaluate/)를 참조하세요.
- 모델을 디스크에 저장하고 복구합니다. [직렬화 및 저장 가이드](https://www.tensorflow.org/guide/keras/save_and_serialize/)를 참조하세요.
- 다중 GPU를 활용하여 모델의 훈련 속도를 향상합니다. [다중 GPU 및 분산 훈련 가이드](distributed_training)를 참조하세요.

## Sequential 모델을 사용한 특성 추출

Sequential 모델이 빌드되면 [Functional API 모델](https://www.tensorflow.org/guide/keras/functional/)처럼 동작합니다. 이는 모든 레이어가 `input` 및 `output` 속성을 갖는다는 것을 의미합니다. 이러한 속성을 사용하면 Sequential 모델 내의 모든 중간 레이어들의 출력을 추출하는 모델을 빠르게 생성하는 등 깔끔한 작업을 수행할 수 있습니다.

In [34]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),                    #  (250-5)//2 + 1
        layers.Conv2D(32, 5, strides=2, activation="relu"),  # (None,250,250,3)(5,5,3,32)+(32,)
        layers.Conv2D(32, 3, activation="relu"),             # (None,123,123,32)(3,3,32,32)+(32,)
        layers.Conv2D(32, 3, activation="relu"),             # (None,121,121,32)(3,3,32,32)+(32,)
                                                             # (None,119,119,32)
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)
print(features)

[<tf.Tensor: shape=(1, 123, 123, 32), dtype=float32, numpy=
array([[[[0.        , 0.54926306, 0.        , ..., 0.        ,
          0.15236846, 0.08554606],
         [0.        , 0.54926306, 0.        , ..., 0.        ,
          0.15236846, 0.08554606],
         [0.        , 0.54926306, 0.        , ..., 0.        ,
          0.15236846, 0.08554606],
         ...,
         [0.        , 0.54926306, 0.        , ..., 0.        ,
          0.15236846, 0.08554606],
         [0.        , 0.54926306, 0.        , ..., 0.        ,
          0.15236846, 0.08554606],
         [0.        , 0.54926306, 0.        , ..., 0.        ,
          0.15236846, 0.08554606]],

        [[0.        , 0.54926306, 0.        , ..., 0.        ,
          0.15236846, 0.08554606],
         [0.        , 0.54926306, 0.        , ..., 0.        ,
          0.15236846, 0.08554606],
         [0.        , 0.54926306, 0.        , ..., 0.        ,
          0.15236846, 0.08554606],
         ...,
         [0.        , 0.5492

다음은 한 레이어에서 특성만 추출하는 것과 유사한 예입니다.

In [ ]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)
# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)  # (None,121,121,32)
print(features)

## Sequential 모델을 통한 전이 학습

전이 학습은 모델에서 맨 아래 레이어를 동결하고 맨 위 레이어만 훈련하는 것으로 구성됩니다. 익숙하지 않은 경우, [전이 학습 가이드](https://www.tensorflow.org/guide/keras/transfer_learning/)를 읽어보세요.

다음은 Sequential 모델과 관련된 두 가지 일반적인 전이 학습 청사진입니다.

먼저 Sequential 모델이 있고 마지막 모델을 제외한 모든 레이어를 동결하려고 한다고 가정합니다. 이 경우 다음과 같이 단순히 `model.layers`를 반복하고 마지막 레이어를 제외하고 각 레이어에서 `layer.trainable = False`를 설정합니다.

```python
model = keras.Sequential([<br>    keras.Input(shape=(784))<br>    layers.Dense(32, activation='relu'),<br>    layers.Dense(32, activation='relu'),<br>    layers.Dense(32, activation='relu'),<br>    layers.Dense(10),<br>])<br> <br># Presumably you would want to first load pre-trained weights.<br>model.load_weights(...)<br> <br># Freeze all layers except the last one.<br>for layer in model.layers[:-1]:<br>  layer.trainable = False<br> <br># Recompile and train (this will only update the weights of the last layer).<br>model.compile(...)<br>model.fit(...)
```

또 다른 일반적인 청사진은 다음과 같이 Sequential 모델을 사용하여 사전 훈련된 모델과 새로 초기화된 분류 레이어를 쌓는 것입니다.

```python
# Load a convolutional base with pre-trained weights<br>base_model = keras.applications.Xception(<br>    weights='imagenet',<br>    include_top=False,<br>    pooling='avg')<br> <br># Freeze the base model<br>base_model.trainable = False<br> <br># Use a Sequential model to add a trainable classifier on top<br>model = keras.Sequential([<br>    base_model,<br>    layers.Dense(1000),<br>])<br> <br># Compile & train<br>model.compile(...)<br>model.fit(...)
```

전이 학습을 한다면 아마도 이 두 가지 패턴을 자주 사용하게 될 것입니다.

이것이 Sequential 모델에 대해 알아야 할 전부입니다!

Keras에서 모델을 빌드하는 방법에 대한 자세한 내용은 다음을 참조하세요.

- [Functional API 가이드](https://www.tensorflow.org/guide/keras/functional/)
- [하위 클래스화를 통한 새 레이어 및 모델 생성 가이드](https://www.tensorflow.org/guide/keras/custom_layers_and_models/)